### Model Category Configuration

In [1]:
# All the features: ['all', 'actual', 'entsoe', 'weather_t', 'weather_i', 'holiday', 'weekday', 'hour', 'month']
model_cat_id = "01"
feature = ['actual', 'entsoe']

# LSTM layer configuration
layer_conf = [ True, True, True]
cells = [[ 5, 10, 20, 30, 50, 75, 100, 125, 150], [0, 10, 20, 50], [0, 10, 15, 20]]
dropout = [0, 0.1, 0.2]
batch_size = [8]
timesteps = [1]

#### Select backend & Check if keras work

In [2]:
import os
# os.environ["KERAS_BACKEND"] = "torch"
# os.environ["KERAS_BACKEND"] = "jax"
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
print(keras.__version__)

3.3.3


In [6]:
import sys
import math
import itertools
import datetime as dt
from decimal import *
import pytz
import time as t
import pandas as pd
import numpy as np
from pandas import read_csv
from numpy import newaxis
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as stattools
from tabulate import tabulate
import math
import keras
from keras import backend as K
from keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

mpl.rcParams['figure.figsize'] = (9, 5)
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from lstm_load import data, lstm

rmse: Root Mean Square Error - 模型预测值与实际值之间的差异
mae: Mean Absolute Error - 衡量预测值与实际值的差异。
mape: Mean Absolute Percentage Error - 衡量预测误差相对于实际值的百分比
train_loss - 存储训练集上的损失值。损失函数是用于衡量模型预测误差的标准。
valid_loss - 存储验证集上的损失值。用于评估模型在未见数据上的表现

In [9]:
path = os.path.join(os.path.abs(''), '../data/fulldataset.csv')
loc_tz = pytz.timezone('Europe/Zurich')
split_date = loc_tz.localize(dt.datetime(2017,2,1,0,0,0,0))
validation_split = 0.2
epochs = 30
verbose = 0
results = pd.DataFrame(columns=['module_name', 'config', 'dropout', 'train_loss', 'train_rmse', 'train_mae', 'train_mape', 'valid_loss', 'valid_rmse', 'valid_mae', 'valid_mape', 'test_rmse', 'test_mae', 'test_mape', 'epochs', 'batch_train', 'input_shape', 'total_time', 'time_step', 'splits'])
early_stopping = True
min_delta = 0.006
patience = 2

AttributeError: module 'ntpath' has no attribute 'abs'

In [ ]:
def generate_combinations(model_name=None, layer_conf=None, cells=None, dropout=None, batch_size=None, timesteps=None):
    models = []
    layer_conb = list(itertools.product(*cells))
    configs = [layer_conb, dropout, batch_size, timesteps]
    combinations = list(itertools.product(*configs))

    for ix, comb in enumerate(combinations):
        m_name = model_name
        m_name += str(ix + 1)

        layers = []
        for idx, level in enumerate(comb[0]):
            return_sequence = True
            if all(size == 0 for size in comb[0][idx + 1:]) == True:
                return_sequence = False
            if (idx + 1) == len(comb[0]):
                return_sequence = False
            if level > 0:
                layers.append({'type': 'lstm', 'cell': level, 'dropout': comb[1], 'statful': layer_conf[idx], 'ret_seq': return_sequence })
                m_name += '_1-' + str(comb[1])
        if comb[1] > 0:
            m_name += '_d-' + str(comb[1])
        model_config = {
            'name': m_name,
            'layers': layers,
            'batch_size': comb[2],
            'timesteps': comb[3]
        }
        models.append(model_config)

        print('==================')
        print(tabulate([
            ['Number of model configs generated', len(combinations)]],
            tablefmt="jira", numalign="right", floatfmt=".3f"))
        return models